In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from tqdm import tqdm
%matplotlib inline

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

import sys
sys.path.append('../')
import utils
import settings

In [ ]:
train_a, train_b, train_c, X_train_estimated_a, X_train_estimated_b, X_train_estimated_c, X_train_observed_a, X_train_observed_b, X_train_observed_c, X_test_estimated_a, X_test_estimated_b, X_test_estimated_c = utils.read_files(diff_path='../')

In [ ]:
X_reshaped_train_estimated_a, X_reshaped_train_estimated_b, X_reshaped_train_estimated_c, X_reshaped_train_observed_a, X_reshaped_train_observed_b, X_reshaped_train_observed_c, X_reshaped_test_estimated_a, X_reshaped_test_estimated_b, X_reshaped_test_estimated_c = utils.get_reshaped_files(diff_path='../')

In [ ]:
X_a_reshaped = pd.concat([X_reshaped_train_observed_a, X_reshaped_test_estimated_a])
X_a_reshaped

In [ ]:
days = utils.get_for_days_to_predict()
len(days)

In [ ]:
30 + 31 + 3 

In [ ]:
keys = [1, 2, 3]
X_reshaped_frame_a = [ X_reshaped_train_estimated_a, X_reshaped_train_observed_a, X_reshaped_test_estimated_a ]
X_frame_a = pd.concat(X_reshaped_frame_a, keys=keys)
X_frame_a.reset_index(level=0, inplace=True, names='frame_type')

train_a = train_a.rename(columns={'time': 'date_forecast_0'})

X_frame_a['date_forecast_0'] = pd.to_datetime(X_frame_a['date_forecast_0'])
train_a['date_forecast_0'] = pd.to_datetime(train_a['date_forecast_0'])

X_y_a = X_frame_a.merge(train_a, on='date_forecast_0', how='inner')
X_y_a.reset_index(drop=True, inplace=True)

In [ ]:
X_y_a2 = X_frame_a.merge(train_a, on='date_forecast_0', how="outer", indicator=True)
X_y_a2.reset_index(drop=True, inplace=True)

In [ ]:
X_train_observed_a

In [ ]:
X_frame_a.describe()

In [ ]:
X_y_a

In [ ]:
# fig, axs = plt.subplots(3, 1, figsize=(20, 10), sharex=True)
# feature_name = 'pv_measurement'
# train_a[['time', feature_name]].set_index('time').plot(ax=axs[0], title='Train A', color='red')
# train_b[['time', feature_name]].set_index('time').plot(ax=axs[1], title='Train B', color='blue')
# train_c[['time', feature_name]].set_index('time').plot(ax=axs[2], title='Train C', color='green')


In [ ]:
date1 = np.datetime64('2020-04-29 08:00:00')
date2 = np.datetime64('2020-05-03 22:00:00')

In [ ]:
important_keys = utils.get_most_important_keys()

In [ ]:
y = np.array(train_a["pv_measurement"])
fft_y = np.fft.fft(y[:29668])
freq_y = np.fft.fftfreq(len(y)) # Spacing in hour
plt.figure(figsize=(10,8))
plt.plot(freq_y[:len(fft_y)//2], np.abs(fft_y)[:len(fft_y)//2])
plt.title("Magn of fft(pv_measurement) on frequency (hours^-1)")
plt.grid()
plt.plot()


In [ ]:
1 / freq_y[np.argmax(np.abs(fft_y[1:len(fft_y)//2]))]

A priori la différence d'échelle est trop grande entre les fréquences pour pouvoir avoir les frequences à l'échelle des mois/années

Pour cela on doit échantilloner y en prenant une mesure par jour ou 2 jours ou semaine etc

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(freq_y[:len(fft_y)//2], 20 * np.log10(np.abs(fft_y))[:len(fft_y)//2])
plt.title("Magn of fft(pv_measurement) on frequency (hours^-1)")
plt.grid()
plt.plot()

In [ ]:
keys2 = important_keys

X = np.array(X_train_observed_a[['clear_sky_energy_1h:J', 'clear_sky_rad:W', 'diffuse_rad:W', 'diffuse_rad_1h:J','direct_rad:W']])
# fft_x = np.fft.fft(X, axis=(0,))
fft_x = np.zeros(X.shape, dtype=complex)
for dim in range(X.shape[1]):
    fft_x[:, dim] = np.fft.fft(X[:, dim])
freq_x = np.fft.fftfreq(118669, .25) # Spacing in quarter hour
plt.figure(figsize=(10,8))
for s in range(fft_x.shape[1]):
    plt.plot(freq_x[:len(freq_x)//2], np.abs(fft_x.T[s])[:len(freq_x)//2])
plt.title("Magn of fft(pv_measurement) on frequency (hours^-1)")
plt.grid()
plt.plot()

In [ ]:
important_keys

In [ ]:
keys2 = important_keys

x = np.array(X_train_observed_a[['clear_sky_energy_1h:J', 'clear_sky_rad:W', 'diffuse_rad:W', 'diffuse_rad_1h:J','direct_rad:W']])
x = np.array([ x[k,:] for k in range(0, len(X), 4)])
fft_x = np.zeros(x.shape, dtype=complex)
for dim in range(x.shape[1]):
    fft_x[:, dim] = np.fft.fft(x[:, dim])
freq_x = np.fft.fftfreq(len(x), .25)

plt.figure(figsize=(10,8))
for s in range(fft_x.shape[1]):
    plt.subplot(5, 1, s + 1)
    plt.plot(freq_x[:len(freq_x)//2], np.abs(fft_x.T[s])[:len(freq_x)//2])
    plt.title("Magn of fft(pv_measurement) on frequency (hours^-1)")
    plt.grid()

plt.subplots_adjust(top=1., hspace=0.5)
plt.plot()

In [ ]:
X_train_observed_a[['clear_sky_energy_1h:J', 'clear_sky_rad:W', 'diffuse_rad:W', 'diffuse_rad_1h:J','direct_rad:W']]

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(20, 10), sharex=True)
feature_name = 'clear_sky_energy_1h:J' # 'pv_measurement'
X_train_observed_a[['date_forecast', feature_name]].set_index('date_forecast').plot(ax=axs[0], title='Train A', color='red')
X_train_observed_a[['date_forecast', feature_name]].set_index('date_forecast').plot(ax=axs[1], title='Train B', color='blue')
X_train_observed_a[['date_forecast', feature_name]].set_index('date_forecast').plot(ax=axs[2], title='Train C', color='green')

In [ ]:
H = np.array([ fft_y / fft_x[:, k] for k in range(fft_x.shape[1]) ]).T
H.shape

In [ ]:
keys2 = important_keys
plt.figure(figsize=(10,8))
for s in range(H.shape[1]):
    plt.subplot(5, 1, s + 1)
    plt.plot(freq_x[:len(freq_x)//2], np.abs(H.T[s])[:len(freq_x)//2])
    plt.title("Magn of fft(pv_measurement) on frequency (hours^-1)")
    plt.grid()

plt.subplots_adjust(top=1., hspace=0.5)
plt.plot()

In [ ]:
h = np.fft.ifft(H)

In [ ]:
plt.plot(h)

In [ ]:
h.shape

In [ ]:
y_pred = np.convolve(h[:,0], x[:,0], mode='full')
plt.plot(y_pred)
plt.show()
for k in range(1, x.shape[1]):
    y_pred += np.convolve(h[:,k], x[:,k], mode='full')
    plt.plot(y_pred)
    plt.show()
plt.show()
plt.plot(y_pred)

In [ ]:
plt.plot(train_a['pv_measurement'])

In [ ]:
# fft_test_estimated_a = np.fft.fft(X_test_estimated_a[])

In [ ]:
x = np.array(X_train_observed_a[['clear_sky_energy_1h:J', 'clear_sky_rad:W', 'diffuse_rad:W', 'diffuse_rad_1h:J','direct_rad:W']])
x = np.array([ x[k,:] for k in range(0, len(x), 4)])

y = np.array(train_a["pv_measurement"])
x.shape, y.shape

In [ ]:
def minmax_norm(df):
    return ( df - np.min(df) ) / ( np.max(df) - np.min(df))

In [ ]:
from sklearn.preprocessing import normalize

In [ ]:
keys2 = important_keys

x = np.array(X_train_observed_a[['clear_sky_energy_1h:J', 'clear_sky_rad:W', 'diffuse_rad:W', 'diffuse_rad_1h:J','direct_rad:W']])
x = np.array([ x[k, :] for k in range(0, len(x), 4)])
# print(x.shape)
for k in range(x.shape[1]):
    print(x[:, k].reshape(-1, 1).shape)
x = np.array( [ normalize(x[:, k].reshape(-1, 1), norm='l2') for k in range(x.shape[1])]).T.reshape(-1, x.shape[1])
# print(x.shape)
y = np.array(train_a["pv_measurement"])[:len(x)]
y = np.array([ normalize(y.reshape(-1, 1), norm='l2') for _ in range(x.shape[1])]).T.reshape(-1, x.shape[1])

# x[:, 0] = np.log(x[:, 0])
# y[:, 0] = 1 - np.exp(- y[:, 0])
x2 = x[:, 0] + np.exp(x[:, 0])

plt.figure(figsize=(10,8))
for s in range(x.shape[1]):
    plt.subplot(5, 1, s + 1)
    sns.scatterplot(x=x[:, s], y=y[:, s]) # , hue='y', data=data_1)
    # plt.plot(x.T[s], y[:, s])
    plt.title(f"pv_measurement on {important_keys[s+1]}")
    plt.grid()

plt.subplots_adjust(top=1., hspace=0.5)
plt.plot()

In [ ]:

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

x_train, x_test, y_train, y_test = train_test_split(x.reshape(-1, 1), y, test_size=0.2, random_state=42)
# Créez un modèle de régression linéaire
model = LinearRegression()

# Entraînez le modèle sur les données d'entraînement
model.fit(x_train, y_train)

# Faites des prédictions sur les données de test
y_pred = model.predict(x_test)

# Évaluez les performances du modèle
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

# Maintenant, vous pouvez utiliser le modèle pour prédire y à partir de nouvelles données x
new_x = np.array([6, 7, 8])
predicted_y = model.predict(new_x.reshape(-1, 1))
print("Predicted y:", predicted_y)

In [ ]:
# plt.plot(np.arctan(predicted_y))

In [ ]:
plt.plot(x[:, 0], y)
plt.show()
plt.plot(x2, y)

In [ ]:
plt.plot(y)

In [ ]:
np.cov(X_train_observed_a['clear_sky_energy_1h:J'][:30000])
x = np.array(X_train_observed_a[['clear_sky_energy_1h:J', 'clear_sky_rad:W', 'diffuse_rad:W', 'diffuse_rad_1h:J','direct_rad:W']])
x = np.array([ x[k,:] for k in range(0, len(x), 4) ])
plt.plot(x[:,0])

In [ ]:
to_plot = (x[:, 0] / np.max(x[:, 0])) * 2 - 1
# toplot2 = ((x[:, 0] / np.max(x[:, 0]))) 
# plt.plot(to_plot - toplot2)
a = np.linspace(0, len(to_plot), len(to_plot))
freq = 1 / ((np.argmax(x[:, 0]) - np.argmax(x[:5000, 0])) / 2)
plt.plot(np.sin(2 * np.pi * freq * (a + (np.arcsin(x[15, 0]) * 2 - 1))) )
# plt.plot(to_plot)


In [ ]:
a + np.arcsin(x[15, 0])

In [ ]:
x[15, 0] / np.max(x[:, 0])